In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import time
from util.parsers import *

In [2]:
cols = [
    #"vehice_key",
    "vehicle_year",
    "vehicle_age",
    "vehicle_number_cylinders",
    "vehicle_mileage",
    "vehicle_make",
    "vehicle_model",
    #"vehicle_body_type",
    #"vehicle_drive_type",
    #"vehicle_usage",
    #"vehicle_condition",
    "vehicle_engine",
    #"vehicle_paint",
    #"vehicle_size",
    "vehicle_fuel_type",
    "vehicle_transmission"
]

In [3]:
vehicle_dim = pd.DataFrame(columns=cols)

In [4]:
vehicle_dim.shape

(0, 9)

In [5]:
PATH = "../stage_1/data/processedData/"

In [6]:
def map_columns(
    df,
    year_label,
    mileage_label,
    make_label, 
    model_label,
    fuel_label,
    cyl_label,
    eng_label,
    trans_label
):
    col_map = {
        year_label: "vehicle_year",
        mileage_label: "vehicle_mileage",
        make_label: "vehicle_make", 
        model_label: "vehicle_model",
        fuel_label: "vehicle_fuel_type",
        cyl_label: "vehicle_number_cylinders",
        #drive_label: "vehicle_drive_type",
        eng_label: "vehicle_engine",
        trans_label: "vehicle_transmission"
    }
    df.columns = df.columns.map(col_map)
    df = df.loc[:, df.columns.notna()]
    return df

- Add *vehicles* dataset to dim

In [7]:
df = pd.read_csv(f"{PATH}/vehicles.csv", index_col=0)
df.head(1)

,id,region,region_url,price,year,manufacturer,model,cylinders,fuel,odometer,title_status,transmission,VIN,drive,type,description,state,lat,long,posting_date
31,7316356412,auburn,https://auburn.craigslist.org,15000,2013.0,ford,f150,6 cylinders,gas,128000.0,clean,automatic,NaN,rwd,truck,2013 F-150 XLT V6 4 Door. Good condition. Leve...,al,32.592,-85.5189,2021-05-03 14:02:03-05:00


In [8]:
v_cols = [
    "year", 
    "manufacturer",
    "model", 
    "cylinders",
    "fuel",
    "odometer",
    "transmission", 
    #"drive"
]
df = df[v_cols]

In [9]:
df.head()

,year,manufacturer,model,cylinders,fuel,odometer,transmission
31,2013.0,ford,f150,6 cylinders,gas,128000.0,automatic
32,2012.0,gmc,sierra 2500,8 cylinders,gas,68696.0,other
33,2016.0,chevrolet,c1500,6 cylinders,gas,29499.0,other
34,2019.0,toyota,tacoma,6 cylinders,gas,43000.0,automatic
35,2016.0,chevrolet,colorado,6 cylinders,gas,17302.0,other


In [10]:
df.shape

(157431, 7)

In [11]:
df = map_columns(df, "year", "odometer", "manufacturer", "model", "fuel", "cylinders", None, "transmission")

In [12]:
df = df.drop_duplicates()

In [13]:
df.shape

(100685, 7)

In [14]:
vehicle_dim = pd.concat((vehicle_dim, df)).reset_index(drop=True)

In [15]:
vehicle_dim.shape

(100685, 9)

- Add *used_cars* dataset

In [16]:
df = pd.read_csv(f"{PATH}/used_car_sales.csv", )
df.head(1)

,ID,pricesold,yearsold,zipcode,Mileage,Make,Model,Year,Engine,BodyType,NumCylinders,DriveType,datesold,lat,long,state,fuel
0,137178,7500,2020,78611,84430,ford,mustang,1988.0,5.0l gas v8,sedan,8,RWD,2020-03-19 00:00:00,30.767327,-98.30109,tx,gas


In [17]:
uc_cols = [
    "Mileage", 
    "Make", 
    "Model", 
    "Year", 
    "Engine", 
    #"BodyType", 
    "NumCylinders", 
    #"DriveType", 
    "fuel"]
df = df[uc_cols]

In [18]:
df = map_columns(df, "Year", "Mileage", "Make", "Model", "fuel", "NumCylinders", "Engine", None)

In [19]:
df.shape

(72590, 7)

In [20]:
df.head()

,vehicle_mileage,vehicle_make,vehicle_model,vehicle_year,vehicle_engine,vehicle_number_cylinders,vehicle_fuel_type
0,84430,ford,mustang,1988.0,5.0l gas v8,8,gas
1,40703,porsche,911,2002.0,3.6l,6,gas
2,71300,mercury,montclair,1965.0,NaN,0,gas
3,15000,pontiac,gto,1970.0,NaN,0,gas
4,51674,jeep,wrangler,2015.0,3.6l flexible v6,6,gas


In [21]:
vehicle_dim = pd.concat((vehicle_dim, df)).reset_index(drop=True).drop_duplicates()

In [22]:
vehicle_dim.shape

(172357, 9)

- ADD *tn_mvr* dataset

In [23]:
df = pd.read_csv(f"{PATH}/tn_mvr.csv", index_col=0)
df.head(1)

,vin,price,odometer_type,mileage,county,zip,model_year,make,model,vehicle_type,new_used,title_issue_date,purchase_date,lat,long,state,fuel
1013,137ZA8434TE173571,31000.0,1,0.0,Tipton,38053,1996,am-general,hummer,AUTO,U,2019-01-17,2019-01-04,35.347965,-89.90668,tn,gas


In [24]:
df.vehicle_type.unique()

array(['AUTO', 'OTHER', 'TRUCK', 'BUS - COMMERCIAL', 'MOTORCYCLE',
       'SCHOOL BUS'], dtype=object)

In [25]:
tn_cols = [
    "mileage", 
    "model_year", 
    "make", 
    "model", 
    #"new_used",
    "fuel"]
df = df[tn_cols]

In [26]:
df.shape

(535786, 5)

In [27]:
df = map_columns(df , "model_year", "mileage", "make", "model", "fuel", None, None, None)

In [28]:
df = df.drop_duplicates()

In [29]:
vehicle_dim = pd.concat((vehicle_dim, df), axis=0)

In [30]:
vehicle_dim.shape

(360479, 9)

In [31]:
vehicle_dim = vehicle_dim.reset_index(drop=True).drop_duplicates()

- reviews

In [32]:
df = pd.read_csv(PATH + "review.csv", index_col=0)

In [33]:
df.head(1)

,Company,Model,Year,Reviewer,Date,Title,Rating,Review
0,acura,ilx,2013,mahowald,2012-08-12,Getting 43 Mpg on daily commute,5,"I love this car.\r\nGas mileage, suspension, a..."


In [34]:
c = [
    "Year", 
    "Company", 
    "Model", 
    #"new_used",
#    "fuel"
]
df = df[c]

In [35]:
df.shape

(262923, 3)

In [36]:
df = map_columns(df , "Year", None, "Company", "Model", None, None, None, None)

In [37]:
df = df.drop_duplicates()
df.shape

(4319, 3)

In [38]:
vehicle_dim = pd.concat((vehicle_dim, df), axis=0)

In [39]:
vehicle_dim.shape

(364798, 9)

In [40]:
vehicle_dim = vehicle_dim.reset_index(drop=True).drop_duplicates()
vehicle_dim.shape

(364798, 9)

- calculate age

In [41]:
from datetime import datetime

In [42]:
vehicle_dim["vehicle_age"] = datetime.now().year - vehicle_dim.vehicle_year

In [43]:
vehicle_dim.head()

,vehicle_year,vehicle_age,vehicle_number_cylinders,vehicle_mileage,vehicle_make,vehicle_model,vehicle_engine,vehicle_fuel_type,vehicle_transmission
0,2013.0,10.0,6 cylinders,128000.0,ford,f150,NaN,gas,automatic
1,2012.0,11.0,8 cylinders,68696.0,gmc,sierra 2500,NaN,gas,other
2,2016.0,7.0,6 cylinders,29499.0,chevrolet,c1500,NaN,gas,other
3,2019.0,4.0,6 cylinders,43000.0,toyota,tacoma,NaN,gas,automatic
4,2016.0,7.0,6 cylinders,17302.0,chevrolet,colorado,NaN,gas,other


In [44]:
vehicle_dim.vehicle_fuel_type = vehicle_dim.vehicle_fuel_type.map(fuel_map)

In [45]:
vehicle_dim.vehicle_mileage.quantile(0.20)

25739.0

In [46]:
vehicle_dim.vehicle_mileage.quantile(1)

1319891701.0

In [47]:
def mileage_to_miles_band(mile):
    if mile!=mile:
        return np.nan
    if mile < 26000:
        return "[0, 26000)"
    if mile < 60500:
        return "[26000, 60500)"
    if mile < 98500:
        return "[60500, 98500)"
    if mile < 143400:
        return "[98500, 143400)"
    if mile < 209000:
        return "[143400, 209000)"
    return "[209000, ?)"
    

In [48]:
vehicle_dim["vehicle_mileage"] = vehicle_dim.vehicle_mileage.apply(mileage_to_miles_band)

In [49]:
vehicle_dim = vehicle_dim.drop_duplicates()

In [50]:
vehicle_dim.shape

(107890, 9)

In [51]:
vehicle_dim.fillna("Unknown", inplace=True)

In [54]:
vehicle_dim[vehicle_dim.vehicle_mileage=="Unknown"]

,vehicle_year,vehicle_age,vehicle_number_cylinders,vehicle_mileage,vehicle_make,vehicle_model,vehicle_engine,vehicle_fuel_type,vehicle_transmission
360479,2013.0,10.0,Unknown,Unknown,acura,ilx,Unknown,Unknown,Unknown
360480,2014.0,9.0,Unknown,Unknown,acura,ilx,Unknown,Unknown,Unknown
360481,2015.0,8.0,Unknown,Unknown,acura,ilx,Unknown,Unknown,Unknown
360482,2016.0,7.0,Unknown,Unknown,acura,ilx,Unknown,Unknown,Unknown
360483,2017.0,6.0,Unknown,Unknown,acura,ilx,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...
364793,2010.0,13.0,Unknown,Unknown,volvo,v70,Unknown,Unknown,Unknown
364794,2005.0,18.0,Unknown,Unknown,volvo,c70,Unknown,Unknown,Unknown
364795,2014.0,9.0,Unknown,Unknown,volvo,c70,Unknown,Unknown,Unknown
364796,2015.0,8.0,Unknown,Unknown,volvo,c70,Unknown,Unknown,Unknown


In [52]:
vehicle_dim.to_csv("data/vehicleDim.csv", index_label="vehicle_key")